In [1]:
%load_ext autoreload
%autoreload 2

In [189]:
import pandas as pd 
import numpy as np
import json
from datetime import datetime 

from api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap, threading
from preproc import filters

In [111]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2016, 8, 9), datetime(2022, 8, 9)]
    )

In [112]:
#get tickers that are valid across the past six years
tker_dict = val.get_tkers(settings)
valid_lst = val.get_valid_tkers(tker_dict, pretty_print = False)

In [192]:
#filtering the tickers we want for analysis 
raw_df = pd.DataFrame(valid_lst, columns = ["ticker", "name", "exchange", "type", "date", "delist", "status"])
type_filt = "Stock" 
exchange_filt = "NYSE"
raw_df = raw_df.query("type == @type_filt")
raw_df = raw_df.query("exchange == @exchange_filt")
display(raw_df)

#list of tickers to work on for now
temp_lst = raw_df["ticker"].tolist()[:100]

,ticker,name,exchange,type,date,delist,status
5,BGSF,BGSF Inc,NYSE,Stock,2014-04-29,null,Active
11,GS-P-J,Goldman Sachs Group Inc,NYSE,Stock,2013-04-22,null,Active
13,TKC,Turkcell Iletisim Hizmetleri A.S.,NYSE,Stock,2000-07-11,null,Active
16,DRH,Diamondrock Hospitality Company,NYSE,Stock,2005-05-26,null,Active
18,GHL,Greenhill & Co Inc,NYSE,Stock,2004-05-06,null,Active
...,...,...,...,...,...,...,...
5570,JNPR,Juniper Networks Inc,NYSE,Stock,1999-06-25,null,Active
5578,AME,Ametek Inc,NYSE,Stock,1984-07-19,null,Active
5581,DEO,Diageo plc,NYSE,Stock,1996-05-13,null,Active
5587,SMFG,Sumitomo Mitsui Financial Group Inc,NYSE,Stock,2006-01-27,null,Active


In [193]:
#get latest ticker info
ticker_info_url = wrap.get_general_url(settings, ticker_lst = temp_lst, output = "lst", datatype = "csv", function = "OVERVIEW")
ticker_info_data = threading.thrd_json_data(ticker_info_url, key_lst = ["MarketCapitalization"], wrapper_function = wrap.get_json_data)

Total time elapsed: 4.242727041244507 seconds


In [221]:
import requests, json
url = 'https://www.alphavantage.co/query?function=OVERVIEW&output=json&symbol=IBM&apikey=MD27QTTVAK3AJBUQ'
r = requests.get(url)
data = r.json()
#print(data.keys())
lst = ['FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta']

['FiscalYearEnd',
 'LatestQuarter',
 'MarketCapitalization',
 'EBITDA',
 'PERatio',
 'PEGRatio',
 'BookValue',
 'DividendPerShare',
 'DividendYield',
 'EPS',
 'RevenuePerShareTTM',
 'ProfitMargin',
 'OperatingMarginTTM',
 'ReturnOnAssetsTTM',
 'ReturnOnEquityTTM',
 'RevenueTTM',
 'GrossProfitTTM',
 'DilutedEPSTTM',
 'QuarterlyEarningsGrowthYOY',
 'QuarterlyRevenueGrowthYOY',
 'AnalystTargetPrice',
 'TrailingPE',
 'ForwardPE',
 'PriceToSalesRatioTTM',
 'PriceToBookRatio',
 'EVToRevenue',
 'EVToEBITDA',
 'Beta']

In [194]:
daily_urls = wrap.get_general_url(settings, datatype = "csv", ticker_lst = temp_lst, output = "lst", function = "TIME_SERIES_DAILY_ADJUSTED")
daily_data = threading.thrd_csv_data(daily_urls)

Total time elapsed: 25.095728874206543 seconds


In [211]:
testing = filters.daily_filter(settings, daily_data)
display(testing["BGSF"])

,timestamp,adjusted_close,volume
0,2022-08-09,12.368331,22943
1,2022-08-08,12.625387,33562
2,2022-08-05,12.902216,21231
3,2022-08-04,13.129612,42283
4,2022-08-03,12.724255,12741
...,...,...,...
1490,2016-08-16,13.088580,65468
1491,2016-08-15,13.313469,96543
1492,2016-08-12,13.171039,101084
1493,2016-08-10,13.950657,154265


In [217]:
from preproc import make_dfs as df

pivot = df.pivot_data(testing)
display(pivot)

,1,2,3,4,5,6,7,8,9,10,...,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990
0,,,,,,,,,,,,,,,,,,,,,
SUPV,1.710000,1.790000,1.810000,1.680000,1.700000,1.640000,1.690000,1.720000,1.670000,1.580000,...,195692,85849,320715,436921,43803,110219,278999,228546,74152,342065
GS-P-J,24.310026,24.457957,24.477681,24.645336,24.694647,24.714371,24.783405,24.734095,24.655198,24.349474,...,73827,53553,66223,82248,63611,79565,101893,79444,148313,153219
FPI,14.519912,14.519912,14.141737,14.121833,14.778663,14.778663,14.878183,14.768711,14.450248,14.340777,...,98178,108093,159272,59952,65814,156703,140746,58422,50396,46923
SO,77.574746,76.523893,76.603203,77.297162,76.593289,75.274766,76.018294,76.226482,75.185543,72.627807,...,3655715,4830885,6552814,7694959,21767980,5030157,4124547,3190805,5836874,4201527
TKC,2.390000,2.490000,2.370000,2.360000,2.360000,2.380000,2.400000,2.390000,2.350000,2.380000,...,215640,267690,144818,234982,134990,155717,135193,172215,293123,179163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RDY,52.910000,53.170000,52.660000,52.440000,52.090000,51.380000,51.480000,51.124181,51.948125,54.003019,...,270752,81019,126623,170494,589004,108743,88848,143069,212704,290566
SR,74.018663,72.711403,71.612116,72.453912,72.978797,73.414550,74.394995,74.513837,73.681944,71.473467,...,117515,119042,173809,203301,136745,258018,220969,135428,189415,165287
CPA,71.950000,73.240000,72.640000,73.090000,69.290000,67.040000,67.860000,67.220000,67.040000,66.380000,...,397717,330063,618947,526402,965264,685789,435418,598943,398252,453241


,0,1,2,3,4,5,6,7,8,9,...,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990
0,SUPV,1.710000,1.790000,1.810000,1.680000,1.700000,1.640000,1.690000,1.720000,1.670000,...,195692,85849,320715,436921,43803,110219,278999,228546,74152,342065
1,GS-P-J,24.310026,24.457957,24.477681,24.645336,24.694647,24.714371,24.783405,24.734095,24.655198,...,73827,53553,66223,82248,63611,79565,101893,79444,148313,153219
2,FPI,14.519912,14.519912,14.141737,14.121833,14.778663,14.778663,14.878183,14.768711,14.450248,...,98178,108093,159272,59952,65814,156703,140746,58422,50396,46923
3,SO,77.574746,76.523893,76.603203,77.297162,76.593289,75.274766,76.018294,76.226482,75.185543,...,3655715,4830885,6552814,7694959,21767980,5030157,4124547,3190805,5836874,4201527
4,TKC,2.390000,2.490000,2.370000,2.360000,2.360000,2.380000,2.400000,2.390000,2.350000,...,215640,267690,144818,234982,134990,155717,135193,172215,293123,179163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,RDY,52.910000,53.170000,52.660000,52.440000,52.090000,51.380000,51.480000,51.124181,51.948125,...,270752,81019,126623,170494,589004,108743,88848,143069,212704,290566
96,SR,74.018663,72.711403,71.612116,72.453912,72.978797,73.414550,74.394995,74.513837,73.681944,...,117515,119042,173809,203301,136745,258018,220969,135428,189415,165287
97,CPA,71.950000,73.240000,72.640000,73.090000,69.290000,67.040000,67.860000,67.220000,67.040000,...,397717,330063,618947,526402,965264,685789,435418,598943,398252,453241
98,PBH,57.230000,59.400000,59.350000,58.970000,59.540000,60.120000,61.120000,60.310000,60.770000,...,423263,367521,281592,381978,366495,383256,701508,1154010,766753,541916


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#PCA logic flow 

#Step 1: get the standardised features
x = df (with the features from the dataframe)
# Standardizing the features
x = StandardScaler().fit_transform(x)

#Step 2: feeed standardadised features into the PCA
#We can experiment with different PCA n later? 
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ["pc1", "pc2"])

#Step 3: Check if explained variance is good enough
print(pca.explained_variance_ratio_)

#Step 4: 
Visualisation for slides idk 

In [117]:
daily_data.keys()

dict_keys(['SUPV', 'GS-P-J', 'BGSF', 'FPI', 'JBT', 'TKC', 'DRH', 'GHL', 'UNFI', 'SRE', 'MCS', 'SO', 'BML-P-H', 'USB-P-A', 'REX', 'EGP', 'SJW', 'MTRN', 'ETD', 'MUFG', 'PB', 'GJR', 'GDOT', 'TFC', 'SNA', 'RGA', 'F', 'IBM', 'LTC', 'YRD', 'BC', 'BSBR', 'GSK', 'MTN', 'CHU', 'NUS', 'RZB', 'PII', 'IDA', 'LN', 'GNL', 'J', 'AWI', 'PRG', 'SOL', 'DRD', 'FNF', 'WTI', 'IRS', 'C'])

KeyError: "None of ['timestamp'] are in the columns"

In [91]:
import requests, json

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'IBM', '3. Last Refreshed': '2022-10-27', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2022-10-27': {'1. open': '135.55', '2. high': '136.4', '3. low': '134.445', '4. close': '134.77', '5. adjusted close': '134.77', '6. volume': '3993168', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-10-26': {'1. open': '133.72', '2. high': '135.863', '3. low': '132.81', '4. close': '135.01', '5. adjusted close': '135.01', '6. volume': '5140023', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-10-25': {'1. open': '132.0', '2. high': '133.3', '3. low': '131.3', '4. close': '132.93', '5. adjusted close': '132.93', '6. volume': '5957623', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-10-24': {'1. open': '130.9', '2. high': '133.11', '3. low': '129.85', '4. close': '132.69', '5. adjusted close': '132

In [218]:
import requests, json
url = 'https://www.alphavantage.co/query?function=OVERVIEW&output=json&symbol=IBM&apikey=MD27QTTVAK3AJBUQ'
r = requests.get(url)
data = r.json()
print(data.keys())

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])
